<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-19 10:22:21
-------------------
qualified stocks: 94
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  1.01 L
Current:  1.37 C
-------------------
Today PnL: 70.50 K (0.52%)
Current PnL: -18.04 L (-12.27%)
CY Booked + Current PnL: -9.77 L (-6.64%)
-------------------
Total profit:  3.64 L
Total loss:  -21.68 L
-------------------
Total Booked + Current PnL: 17.21 L (14.34%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.04%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 77.66 L (56.75%)
Deployed:  1.20 C
Current:  1.37 C
CAGR/XIRR %: 8.38%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LALPATHLAB,2838.53,3545.00,-20.67,H-SC,11.77,236871.0,43851.0,4193.0,1.60,22.72,1.77,24.89,137.0,10.46,1.74,51.37,X40N,NTT,HEALTHCARE
36,HINDALCO,651.95,756.01,-17.42,H-LC,6.82,113656.0,9344.0,7308.0,-0.53,8.96,6.43,15.96,9.0,1.28,0.84,27.37,X5K,ATH,METALS
69,SBIN,760.30,863.00,-11.34,M-LC,3.89,219592.0,18112.0,9113.0,0.12,8.99,4.15,13.51,60.0,1.99,1.62,22.72,XY25,NTT,BANKS
4,APOLLOHOSP,7098.95,8285.00,-13.64,H-LC,6.60,172458.0,16281.0,9813.0,0.34,10.42,5.69,16.71,22.0,1.66,1.27,29.68,X40N,BTT,HEALTHCARE
56,MEDANTA,1087.93,1486.00,11.67,H-MC,13.96,159430.0,33230.0,12946.0,2.32,26.33,8.12,36.59,108.0,2.57,1.17,42.14,XY24,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,RAJOOENG,102.50,143.33,-32.03,H-SC,7.58,103120.0,620.0,40206.0,1.47,0.60,38.99,39.83,114.0,0.02,0.76,2.10,AR,ATH,MISC
12,BANKINDIA,116.91,190.00,-30.43,H-MC,7.52,178024.0,-1784.0,114202.0,0.78,-0.99,64.15,62.52,88.0,-0.02,1.31,32.76,XR,NTT,BANKS
59,PGHH,13388.00,18062.58,-31.21,H-MC,6.12,199125.0,-1695.0,71804.0,-0.77,-0.84,36.06,34.92,80.0,-0.02,1.47,3.79,X40,ATH,FMCG
57,NATIONALUM,189.63,247.44,-49.46,H-MC,1.97,100695.0,191.0,30450.0,0.88,0.19,30.24,30.49,79.0,0.01,0.74,32.66,MH,ATH,METALS
93,WIPRO,243.46,420.00,-13.34,M-LC,7.63,152030.0,1085.0,108367.0,-0.65,0.72,71.28,72.51,53.0,0.01,1.12,6.78,XR,NTT,IT


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,ITC,411.90,452.0,-38.34,H-LC,2.81,147893.0,-2039.0,16638.0,-1.25,-1.36,11.25,9.74,4.0,-0.12,1.09,4.79,X40,NTT,FMCG
21,CIPLA,1492.70,1795.0,-19.78,H-LC,3.10,213898.0,9398.0,32021.0,-0.25,4.60,14.97,20.25,10.0,0.29,1.57,13.73,X40N,BTT,PHARMA
58,NESTLEIND,1134.30,1377.0,-13.70,H-LC,3.66,265473.0,47.0,56758.0,-0.84,0.02,21.38,21.40,11.0,0.00,1.95,7.35,XY25,NTT,FMCG
54,LTIM,5564.16,7230.2,-8.09,H-LC,3.75,182448.0,-17862.0,77832.0,-0.42,-8.92,42.66,29.94,16.0,-0.23,1.34,26.04,X200,ATH,IT
5,ASIANPAINT,2961.56,4250.0,-10.58,H-LC,4.18,174440.0,-26946.0,114555.0,-0.85,-13.38,65.67,43.51,27.0,-0.24,1.28,21.43,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,LAOPALA,369.40,464.00,125.48,H-SC,2.10,71662.0,-29184.0,55008.0,7.85,-28.94,76.76,25.61,142.0,-0.53,0.53,32.86,AR,NTT,CERAMICS
92,WHIRLPOOL,1219.98,2270.00,-42.31,M-SC,4.52,97492.0,5994.0,72758.0,4.77,6.55,74.63,86.07,223.0,0.08,0.72,41.03,XR,NTT,DURABLES
84,TTKPRESTIG,769.29,770.00,103.63,M-SC,9.84,87704.0,-13073.0,13164.0,3.95,-12.97,15.01,0.09,245.0,-0.99,0.65,15.22,OX40N,NTT,DURABLES
27,DIXON,14367.00,18940.15,-33.00,H-MC,6.50,134048.0,19112.0,17480.0,3.49,16.63,13.04,31.83,81.0,1.09,0.99,38.94,X40N,ATH,IT
80,TATAMOTORS,761.77,1065.00,-20.18,M-LC,5.24,253084.0,-23439.0,133502.0,3.14,-8.48,52.75,39.81,54.0,-0.18,1.86,21.29,XY24,NTT,AUTO


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,VALIANTORG,512.64,838.00,-322.08,H-SC,15.99,94302.0,-38984.0,123573.0,-2.03,-29.25,131.04,63.47,139.0,-0.32,0.69,58.07,XR,NTT,CHEMICALS
77,SYMPHONY,1306.42,1306.00,-41.14,M-SC,13.88,125511.0,-45630.0,45573.0,-1.99,-26.66,36.31,-0.03,196.0,-1.00,0.92,1.46,OX40N,NTT,DURABLES
47,ITC,411.90,452.00,-38.34,H-LC,2.81,147893.0,-2039.0,16638.0,-1.25,-1.36,11.25,9.74,4.0,-0.12,1.09,4.79,X40,NTT,FMCG
18,CAMPUS,294.86,393.00,-28.59,M-SC,2.92,148794.0,-18981.0,74829.0,-1.10,-11.31,50.29,33.28,210.0,-0.25,1.10,15.20,XY24,NTT,FOOTWEAR
65,ROUTE,1558.40,2242.05,-45.93,H-SC,19.69,58179.0,-44675.0,89793.0,-0.94,-43.44,154.34,43.87,118.0,-0.50,0.43,2.15,SR,ATH,IT


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,TTKPRESTIG,769.29,770.0,103.63,M-SC,9.84,87704.0,-13073.0,13164.0,3.95,-12.97,15.01,0.09,245.0,-0.99,0.65,15.22,OX40N,NTT,DURABLES
43,INDIGOPNTS,1407.73,1408.0,120.42,M-SC,2.26,144448.0,-30111.0,30146.0,2.18,-17.25,20.87,0.02,221.0,-1.00,1.06,25.26,OX40N,NTT,PAINTS
20,CERA,7989.07,9475.0,-17.18,H-SC,3.04,118134.0,-25669.0,52416.0,0.98,-17.85,44.37,18.60,149.0,-0.49,0.87,30.36,OX40N,NTT,CERAMICS
85,UJJIVANSFB,52.77,60.0,83.20,H-SC,16.87,115911.0,-26568.0,46086.0,-0.42,-18.65,39.76,13.70,163.0,-0.58,0.85,36.81,OX40N,NTT,BANKS
50,KANSAINER,299.63,340.0,-68.52,H-SC,3.17,217485.0,-52182.0,88516.0,2.02,-19.35,40.70,13.47,138.0,-0.59,1.60,10.00,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TCS,3794.03,4998.00,-29.01,H-LC,16.19,235248.0,-60686.0,154605.0,0.16,-20.51,65.72,31.73,1.0,-0.39,1.73,0.43,X40,BTT,IT
45,INFY,1461.46,2275.00,-21.03,H-LC,11.58,258279.0,-3322.0,148949.0,0.49,-1.27,57.67,55.67,3.0,-0.02,1.90,4.76,X40,BTT,IT
47,ITC,411.90,452.00,-38.34,H-LC,2.81,147893.0,-2039.0,16638.0,-1.25,-1.36,11.25,9.74,4.0,-0.12,1.09,4.79,X40,NTT,FMCG
90,VBL,492.64,671.64,-10.71,H-LC,9.78,271044.0,6004.0,90285.0,0.80,2.27,33.31,36.33,5.0,0.07,2.00,15.87,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-42.10,H-LC,11.28,196716.0,-19803.0,112718.0,0.13,-9.15,57.30,42.91,7.0,-0.18,1.45,3.02,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LALPATHLAB,2838.53,3545.0,-20.67,H-SC,11.77,236871.0,43851.0,4193.0,1.60,22.72,1.77,24.89,137.0,10.46,1.74,51.37,X40N,NTT,HEALTHCARE
56,MEDANTA,1087.93,1486.0,11.67,H-MC,13.96,159430.0,33230.0,12946.0,2.32,26.33,8.12,36.59,108.0,2.57,1.17,42.14,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-9.64,H-LC,7.14,225674.0,41193.0,29180.0,1.04,22.33,12.93,38.14,38.0,1.41,1.66,35.30,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,MEDANTA,1087.93,1486.00,11.67,H-MC,13.96,159430.0,33230.0,12946.0,2.32,26.33,8.12,36.59,108.0,2.57,1.17,42.14,XY24,NTT,HEALTHCARE
15,BLUESTARCO,1646.70,2326.38,11.47,H-MC,6.23,192450.0,27780.0,40184.0,0.46,16.87,20.88,41.28,89.0,0.69,1.42,26.20,X40N,ATH,AC
51,KPIGREEN,497.21,731.35,11.32,H-SC,11.41,132766.0,7469.0,51540.0,2.28,5.96,38.82,47.09,141.0,0.14,0.98,66.17,MH,ATH,POWER
91,VOLTAS,1278.28,1918.49,-2.20,H-MC,7.62,205140.0,13398.0,82630.0,1.36,6.99,40.28,50.08,99.0,0.16,1.51,14.26,XY25,ATH,AC
35,HEROMOTOCO,4311.81,5949.07,-3.95,H-MC,9.41,177712.0,26799.0,30513.0,1.88,17.76,17.17,37.97,93.0,0.88,1.31,47.56,AR,ATH,AUTO


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-3.95,H-MC,9.41,177712.0,26799.0,30513.0,1.88,17.76,17.17,37.97,93.0,0.88,1.31,47.56,AR,ATH,AUTO
51,KPIGREEN,497.21,731.35,11.32,H-SC,11.41,132766.0,7469.0,51540.0,2.28,5.96,38.82,47.09,141.0,0.14,0.98,66.17,MH,ATH,POWER
42,INDIAMART,2327.09,4850.92,-50.16,H-SC,11.91,139157.0,15821.0,117936.0,2.05,12.83,84.75,108.45,119.0,0.13,1.02,38.05,AR,ATH,MISC
92,WHIRLPOOL,1219.98,2270.00,-42.31,M-SC,4.52,97492.0,5994.0,72758.0,4.77,6.55,74.63,86.07,223.0,0.08,0.72,41.03,XR,NTT,DURABLES
62,RAJOOENG,102.50,143.33,-32.03,H-SC,7.58,103120.0,620.0,40206.0,1.47,0.60,38.99,39.83,114.0,0.02,0.76,2.10,AR,ATH,MISC


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-49.70,L-SC,21.40,74131.0,-39418.0,79468.0,1.42,-34.71,107.20,35.27,268.0,-0.50,0.55,81.44,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-2.82,H-SC,15.49,90515.0,-10448.0,104979.0,0.92,-10.35,115.98,93.63,148.0,-0.10,0.67,34.82,SR,ATH,CHEMICALS
55,MASFIN,326.60,399.50,-18.25,H-SC,12.68,92925.0,-5055.0,26920.0,0.96,-5.16,28.97,22.32,152.0,-0.19,0.68,34.77,XR,ATH,FINANCE
89,VALIANTORG,512.64,838.00,-322.08,H-SC,15.99,94302.0,-38984.0,123573.0,-2.03,-29.25,131.04,63.47,139.0,-0.32,0.69,58.07,XR,NTT,CHEMICALS
9,BAJAJFINSV,1994.80,2758.00,-9.86,H-LC,7.11,96775.0,-970.0,38371.0,-0.93,-0.99,39.65,38.26,35.0,-0.03,0.71,27.39,X40,BTT,FINANCE


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LALPATHLAB,2838.53,3545.00,-20.67,H-SC,11.77,236871.0,43851.0,4193.0,1.60,22.72,1.77,24.89,137.0,10.46,1.74,51.37,X40N,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,-3.95,H-MC,9.41,177712.0,26799.0,30513.0,1.88,17.76,17.17,37.97,93.0,0.88,1.31,47.56,AR,ATH,AUTO
28,DMART,3764.92,5201.00,-9.64,H-LC,7.14,225674.0,41193.0,29180.0,1.04,22.33,12.93,38.14,38.0,1.41,1.66,35.30,X40N,NTT,FMCG
4,APOLLOHOSP,7098.95,8285.00,-13.64,H-LC,6.60,172458.0,16281.0,9813.0,0.34,10.42,5.69,16.71,22.0,1.66,1.27,29.68,X40N,BTT,HEALTHCARE
5,ASIANPAINT,2961.56,4250.00,-10.58,H-LC,4.18,174440.0,-26946.0,114555.0,-0.85,-13.38,65.67,43.51,27.0,-0.24,1.28,21.43,X40,BTT,PAINTS


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.46
1,25,40.95
2,50,70.47


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.61
LC    35.40
MC    25.98
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.15
X40N     15.37
X40      14.09
XY25     11.51
XR        8.84
AR        8.31
OX40N     7.35
X5K       2.21
MH        1.72
X200      1.34
SR        1.10
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.23
H-SC    24.07
H-MC    23.08
M-SC    13.09
M-LC     7.11
M-MC     2.57
L-SC     1.45
L-LC     1.06
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.03,-1.58,32.68
IT,11.55,-18.85,78.20
FINANCE,8.00,-19.89,64.24
BANKS,7.44,-12.96,60.92
MISC,5.73,-18.02,85.67
PAINTS,5.57,-15.18,37.87
HEALTHCARE,5.22,7.45,15.92
ELECTRICAL,4.67,-10.88,58.89
AUTO,4.53,-9.64,57.35


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2814403.0,23
XR,1093470.0,13
AR,1080438.0,10
X40,820293.0,11
X40N,519192.0,12
XY25,486016.0,8
OX40N,458795.0,10
SR,194772.0,2
X5K,138462.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2747830.0,28
M-SC,1466885.0,17
H-MC,1240695.0,18
H-LC,1202044.0,19
M-LC,413076.0,5
M-MC,329537.0,2
L-SC,264254.0,3
L-MC,57916.0,1
L-LC,43426.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,931726.0,6
M-SC,XY24,864618.0,7
H-SC,AR,657504.0,5
H-LC,X40,529100.0,6
H-SC,XR,517666.0,6
H-MC,XY24,483558.0,5
M-MC,XY24,329537.0,2
H-LC,AR,233816.0,2
H-MC,X40,209699.0,4


In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
